In [ ]:
# Install and Load Packages

!pip install -q sagemaker-studio-image-build

In [ ]:
import pandas as pd
import numpy as np
import sagemaker as sage

import boto3
import re
import os

from sklearn.datasets import load_iris
from sagemaker import get_execution_role
from sagemaker.serializers import CSVSerializer

In [ ]:
# Data Preparation

data = load_iris()
df = pd.DataFrame(data=data.data, columns=data.feature_names)
df.insert(loc=0, column='target', value=data.target)
mapping = {0:'setosa', 1:'versicolor', 2:'virginica'}
df['target'] = df['target'].map(mapping)
df.head()

In [ ]:
os.makedirs('data', exist_ok=True)
df.to_csv('data/iris.csv', header=False, index=False)


In [ ]:
# Unzip Deployment Package

# Run the following conda install in case unzip is not found
# !conda install -y -c conda-forge unzip

!unzip container.zip

In [ ]:
# Build Docker Container

%%sh

cd container

chmod +x model/train
chmod +x model/serve

sm-docker build . --repository sagemaker-iris-rule:latest --no-logs

In [ ]:
# Train Model
## Note that this model is trained for demo purposes only - it's not the model that we deploy

prefix = 'sagemaker-iris-rule'
role = get_execution_role()
WORK_DIRECTORY = 'data'
data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

sess = sage.Session()

In [ ]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-iris-rule:latest'.format(account, region)

model = sage.estimator.Estimator(image,
                                 role, instance_count=1, instance_type='ml.c4.2xlarge',
                                 output_path='s3://{}/output'.format(sess.default_bucket()),
                                 sagemaker_session=sess)

file_location = data_location + '/iris.csv'
model.fit(file_location)

In [ ]:
# Deploy Rule

predictor = model.deploy(initial_instance_count=1,
                         instance_type='ml.m4.xlarge',
                         serializer=CSVSerializer())

In [ ]:
# Run Inference on Test Data

tdf = pd.read_csv(file_location, header=None)
tdf.drop(tdf.columns[[0]], axis=1, inplace=True)
tdf = tdf.sample(10)

print(predictor.predict(tdf.values).decode('utf-8'))

In [ ]:
# Cleanup

sess.delete_endpoint(predictor.endpoint_name)